In [6]:
from data_cleaning import DataCleaning
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
import pandas as pd
import re
import numpy as np
import tabula
import os

In [7]:
config_dir = "/Users/imobassey/Desktop/DevOps/Git/multinational-retail-data-centralisation299/"  
db_connector = DatabaseConnector(config_dir)
data_extract = DataExtractor()
data_cleaner = DataCleaning()
api_keys = db_connector.read_api_keys()
number_of_stores = data_extract.list_number_of_stores(api_keys['NUMBER_OF_STORES_ENDPOINT'], api_keys['headers'])
print(f" The number of stores are: ", number_of_stores)



 The number of stores are:  451


In [8]:
stores_df = data_extract.retrieve_stores_data(api_keys['STORE_DETAILS_ENDPOINT'], api_keys['headers'], number_of_stores)
stores_df

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
446,446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,None,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
447,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL
448,448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,None,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe
449,449,"Baumplatz 6\n80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35,2020-10-17,Local,8.70745,DE,Europe


In [9]:
#  Correct the 'continent' column
stores_df['continent'] = stores_df['continent'].replace({'eeEurope': 'Europe'})
valid_continents = ['Europe', 'America']
stores_df['continent'] = stores_df['continent'].apply(lambda x: x if x in valid_continents else np.nan)

# Removing rows with non-numeric longitude and latitude
stores_df['longitude'] = pd.to_numeric(stores_df['longitude'], errors='coerce')
stores_df['latitude'] = pd.to_numeric(stores_df['latitude'], errors='coerce')

#  Handle missing values (fill None with np.nan)
stores_df = stores_df.replace('N/A', np.nan).replace('', np.nan)
stores_df = stores_df.applymap(lambda x: np.nan if x in ['K0ODETRLS3', 'K8CXLZDP07', 'UXMWDMX1LC', '3VHFDNP8ET', '9D4LK7X4LZ', 'D23PCWSM6S', '36IIMAQD58', 'NN04B3F6UQ', 'JZP8MIJTPZ', 'B3EH2ZGQAV', '1WZB1TE1HL'] else x)

#  Ensure correct data types
stores_df['staff_numbers'] = pd.to_numeric(stores_df['staff_numbers'], errors='coerce')
stores_df['opening_date'] = pd.to_datetime(stores_df['opening_date'], errors='coerce')

# Replace newline character in the 'address' column with an underscore
stores_df['address'] = stores_df['address'].str.replace("\n", "_")




/var/folders/yq/r9wcdltx4655gh042jpyr47c0000gn/T/ipykernel_45965/2221642985.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stores_df = stores_df.applymap(lambda x: np.nan if x in ['K0ODETRLS3', 'K8CXLZDP07', 'UXMWDMX1LC', '3VHFDNP8ET', '9D4LK7X4LZ', 'D23PCWSM6S', '36IIMAQD58', 'NN04B3F6UQ', 'JZP8MIJTPZ', 'B3EH2ZGQAV', '1WZB1TE1HL'] else x)


In [10]:
stores_df

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,NaN,NaN,NaN,NaN,WEB-1388012W,325.0,2010-06-12,Web Portal,NaN,GB,Europe
1,1,"Flat 72W_Sally isle_East Deantown_E7B 8EB, Hig...",51.62907,None,High Wycombe,HI-9B97EE4E,34.0,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5_50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92.0,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel_South Lydia_WC9 2BE, Westbury",51.26000,None,Westbury,WE-1DE82CEE,69.0,2014-01-02,Super Store,-2.18750,GB,Europe
4,4,"Studio 6_Stephen landing_South Simon_B77 2WA, ...",53.02330,None,Belper,BE-18074576,35.0,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
446,446,"Täschestraße 25_39039 Nördlingen, Kirchlengern",52.20000,None,Kirchlengern,KI-78096E8C,61.0,2005-05-12,Super Store,8.63333,DE,Europe
447,447,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
448,448,"Studio 8_Moss mall_West Linda_M0E 6XR, High Wy...",51.62907,None,High Wycombe,HI-EEA7AE62,33.0,1998-05-14,Local,-0.74934,GB,Europe
449,449,"Baumplatz 6_80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35.0,2020-10-17,Local,8.70745,DE,Europe
